In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np 
import os
import glob
#import cv2
#from libtiff import TIFF

Using TensorFlow backend.


In [2]:
z = '14'
tile_base_path = '../media/combined_37_UDB/%s' % z

In [3]:
photo_dirs = os.listdir(tile_base_path)
train_dirs = photo_dirs[:int(len(photo_dirs) * 0.2)]
test_dirs = photo_dirs[int(len(photo_dirs) * 0.8):]

In [4]:
class myAugmentation(object):

    """
    A class used to augmentate image
    Firstly, read train image and label seperately, and then merge them together for the next process
    Secondly, use keras preprocessing to augmentate image
    Finally, seperate augmentated image apart into train image and label
    """

    def __init__(self, train_path="train", label_path="label", merge_path="merge", aug_merge_path="aug_merge", aug_train_path="aug_train", aug_label_path="aug_label", img_type="tif"):

        """
        Using glob to get all .img_type form path
        """

        self.train_imgs = glob.glob(train_path+"/*."+img_type)
        self.label_imgs = glob.glob(label_path+"/*."+img_type)
        self.train_path = train_path
        self.label_path = label_path
        self.merge_path = merge_path
        self.img_type = img_type
        self.aug_merge_path = aug_merge_path
        self.aug_train_path = aug_train_path
        self.aug_label_path = aug_label_path
        self.slices = len(self.train_imgs)
        self.datagen = ImageDataGenerator(
                                    rotation_range=0.2,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    zoom_range=0.05,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

    def Augmentation(self):

        """
        Start augmentation.....
        """
        trains = self.train_imgs
        labels = self.label_imgs
        path_train = self.train_path
        path_label = self.label_path
        path_merge = self.merge_path
        imgtype = self.img_type
        path_aug_merge = self.aug_merge_path
        if len(trains) != len(labels) or len(trains) == 0 or len(trains) == 0:
            print("trains can't match labels")
            return 0
        for i in range(len(trains)):
            img_t = load_img(path_train+"/"+str(i)+"."+imgtype)
            img_l = load_img(path_label+"/"+str(i)+"."+imgtype)
            x_t = img_to_array(img_t)
            x_l = img_to_array(img_l)
            x_t[:,:,2] = x_l[:,:,0]
            img_tmp = array_to_img(x_t)
            img_tmp.save(path_merge+"/"+str(i)+"."+imgtype)
            img = x_t
            img = img.reshape((1,) + img.shape)
            savedir = path_aug_merge + "/" + str(i)
            if not os.path.lexists(savedir):
                os.mkdir(savedir)
            self.doAugmentate(img, savedir, str(i))


    def doAugmentate(self, img, save_to_dir, save_prefix, batch_size=1, save_format='tif', imgnum=30):

        """
        augmentate one image
        """
        datagen = self.datagen
        i = 0
        for batch in datagen.flow(img,
                          batch_size=batch_size,
                          save_to_dir=save_to_dir,
                          save_prefix=save_prefix,
                          save_format=save_format):
            i += 1
            if i > imgnum:
                break

    def splitMerge(self):

        """
        split merged image apart
        """
        path_merge = self.aug_merge_path
        path_train = self.aug_train_path
        path_label = self.aug_label_path
        for i in range(self.slices):
            path = path_merge + "/" + str(i)
            train_imgs = glob.glob(path+"/*."+self.img_type)
            savedir = path_train + "/" + str(i)
            if not os.path.lexists(savedir):
                os.mkdir(savedir)
            savedir = path_label + "/" + str(i)
            if not os.path.lexists(savedir):
                os.mkdir(savedir)
            for imgname in train_imgs:
                midname = imgname[imgname.rindex("/")+1:imgname.rindex("."+self.img_type)]
                img = cv2.imread(imgname)
                img_train = img[:,:,2]#cv2 read image rgb->bgr
                img_label = img[:,:,0]
                cv2.imwrite(path_train+"/"+str(i)+"/"+midname+"_train"+"."+self.img_type,img_train)
                cv2.imwrite(path_label+"/"+str(i)+"/"+midname+"_label"+"."+self.img_type,img_label)

    def splitTransform(self):

        """
        split perspective transform images
        """
        #path_merge = "transform"
        #path_train = "transform/data/"
        #path_label = "transform/label/"
        path_merge = "deform/deform_norm2"
        path_train = "deform/train/"
        path_label = "deform/label/"
        train_imgs = glob.glob(path_merge+"/*."+self.img_type)
        for imgname in train_imgs:
            midname = imgname[imgname.rindex("/")+1:imgname.rindex("."+self.img_type)]
            img = cv2.imread(imgname)
            img_train = img[:,:,2]#cv2 read image rgb->bgr
            img_label = img[:,:,0]
            cv2.imwrite(path_train+midname+"."+self.img_type,img_train)
            cv2.imwrite(path_label+midname+"."+self.img_type,img_label)

In [5]:
class dataProcess(object):

    def __init__(self, out_rows, out_cols, data_path = "../media/combined_37_UDB/14/", label_path = "../media/combined_37_UDB/14/", 
                 train_dirs = train_dirs, test_dirs = test_dirs, test_path =  "../media/combined_37_UDB/14/", npy_path = "../media/combined_37_UDB/14/", img_type = "png"):

        """

        """

        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.img_type = img_type
        self.test_path = test_path
        self.npy_path = npy_path

    def create_train_data(self):
        print('-'*30)
        print('Creating training images...')
        print('-'*30)
        imgs = []
        for train_dir in train_dirs:
            imgs.extend([x for x in glob.glob("../media/combined_37_UDB/14/%s" % train_dir +"/*."+"png") if 'mask' not in x])
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs), 256, 256, 1), dtype=np.uint8)
        imglabels = np.ndarray((len(imgs), 256, 256,1), dtype=np.uint8) # buildings
        
        i = 0
        for imgname in imgs:
            midname = imgname[imgname.rindex("/")+1:]
            img = load_img(imgname,grayscale = True)
            try:
                label = load_img(imgname.split('\\')[0]  + '/' + imgname.split('\\')[1].split('.')[0] + '_mask.png',grayscale = True)
            except FileNotFoundError:
                continue
            img = img_to_array(img)
            label = img_to_array(label)
            
            imgdatas[i] = img
            imglabels[i] = label
            
            i += 1
            
        print('loading done')
        np.save(self.npy_path + '/imgs_train.npy', imgdatas)
        np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')

    def create_test_data(self):
        i = 0
        print('-'*30)
        print('Creating test images...')
        print('-'*30)
        imgs = []
        for test_dir in test_dirs:
            imgs.extend([x for x in glob.glob("../media/combined_37_UDB/14/%s" % test_dir +"/*."+"png") if 'mask' not in x])
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs),256,256,1), dtype=np.uint8)
        
        i = 0
        for imgname in imgs:
            midname = imgname[imgname.rindex("/")+1:]
            img = load_img(imgname,grayscale = True)
            img = img_to_array(img)
            
            imgdatas[i] = img
            
            i += 1

        print('loading done')
        np.save(self.npy_path + '/imgs_test.npy', imgdatas)
        print('Saving to imgs_test.npy files done.')

    def load_train_data(self, tp = 'roads'):
        print('-'*30)
        print('load train images...')
        print('-'*30)
        imgs_train = np.load(self.npy_path+"/imgs_train.npy")
        if tp == 'roads':
            imgs_mask_train = np.load(self.npy_path+"/imgs_mask_train.npy")
        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        imgs_train /= 255
        imgs_mask_train /= 255
        imgs_mask_train[imgs_mask_train > 0.5] = 1
        imgs_mask_train[imgs_mask_train <= 0.5] = 0
        return imgs_train,imgs_mask_train

    def load_test_data(self):
        print('-'*30)
        print('load test images...')
        print('-'*30)
        imgs_test = np.load(self.npy_path+"/imgs_test.npy")
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255
        return imgs_test

In [6]:
#aug = myAugmentation()
#aug.Augmentation()
#aug.splitMerge()
#aug.splitTransform()
mydata = dataProcess(256,256)
mydata.create_train_data()
mydata.create_test_data()
#imgs_train,imgs_mask_train = mydata.load_train_data()
#print imgs_train.shape,imgs_mask_train.shape

------------------------------
Creating training images...
------------------------------
500
loading done
Saving to .npy files done.
------------------------------
Creating test images...
------------------------------
650
loading done
Saving to imgs_test.npy files done.


In [7]:
import os 
import numpy as np
import tensorflow as tf

from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, Convolution2D, core, BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers.core import Lambda
import keras

from keras import backend as K
from keras.backend import binary_crossentropy

# config = tf.ConfigProto()
# config.gpu_options.allocator_type ='BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [6]:
# %run 'data.ipynb'

ERROR:root:File `"'data.ipynb'.py"` not found.


In [8]:
def slice_batch(x, n_gpus, part):
    """
    Divide the input batch into [n_gpus] slices, and obtain slice no.
    [part].

    i.e. if len(x)=10, then slice_batch(x, 2, 1) will return x[5:].

    """

    sh = K.shape(x)

    L = sh[0] / n_gpus
    L = tf.cast(L, tf.int32)

    if part == n_gpus - 1:

        return x[part*L:]

    return x[part*L:int(part+1)*L]

def to_multi_gpu(model, n_gpus=4):

    """Given a keras [model], return an equivalent model which parallelizes

    the computation over [n_gpus] GPUs.



    Each GPU gets a slice of the input batch, applies the model on that
    slice

    and later the outputs of the models are concatenated to a single
    tensor,

    hence the user sees a model that behaves the same as the original.

    """

    with tf.device('/cpu:0'):

        x = Input(model.input_shape[1:], name=model.input_names[0])


    towers = []

    for g in range(n_gpus):

        with tf.device('/gpu:' + str(g)):

            slice_g = Lambda(slice_batch, lambda shape: shape, arguments={'n_gpus':n_gpus, 'part':g})(x)

            towers.append(model(slice_g))


    with tf.device('/cpu:0'):

        merged = merge(towers, mode='concat', concat_axis=0)

    new_model = Model(input=[x], output=merged)
    
    funcType = type(model.save)
    # monkeypatch the save to save just the underlying model
    def new_save(self_,filepath, overwrite=True):
        model.save(filepath, overwrite)
    new_model.save=funcType(new_save, new_model)
   
    return new_model

In [9]:
smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + binary_crossentropy(y_pred, y_true)


In [12]:
class myUnet(object):

    def __init__(self, tp = 'roads', img_rows = 256, img_cols = 256):

        self.type = tp
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):

        mydata = dataProcess(self.img_rows, self.img_cols)
        imgs_train, imgs_mask_train = mydata.load_train_data(self.type)
        imgs_test = mydata.load_test_data()
        return imgs_train, imgs_mask_train, imgs_test

    #Define the neural network
    def get_unet(self):
        inputs = Input((256, 256, 1))
        #
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(inputs)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv1)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        #
        conv2 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool1)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same', kernel_initializer = 'he_normal')(conv2)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        #
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool2)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv3)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        #
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool3)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv4)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
        drop4 = Dropout(0.5)(conv4)
#         pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
        #
#         conv5 = Convolution2D(512, 3, 3, border_mode='same', kernel_initializer='he_normal')(pool4)
#         conv5 = BatchNormalization(mode=0, axis=1)(conv5)
#         conv5 = keras.layers.advanced_activations.ELU()(conv5)
#         conv5 = Convolution2D(512, 3, 3, border_mode='same', kernel_initializer='he_normal')(conv5)
#         conv5 = BatchNormalization(mode=0, axis=1)(conv5)
#         conv5 = keras.layers.advanced_activations.ELU()(conv5)
#         drop5 = Dropout(0.5)(conv5)
        #
#         up1 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=-1)
#         conv6 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up1)
#         conv6 = BatchNormalization(mode=0, axis=1)(conv6)
#         conv6 = keras.layers.advanced_activations.ELU()(conv6)
#         conv6 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv6)
#         conv6 = BatchNormalization(mode=0, axis=1)(conv6)
#         conv6 = keras.layers.advanced_activations.ELU()(conv6)
        #
        up1 = merge([UpSampling2D(size=(2, 2))(conv4), conv3], mode='concat', concat_axis=-1)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up1)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv7)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        #
        up3 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=-1)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(up3)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(conv8)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        #
        up4 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=-1)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up4)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(2, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        #
        conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

        model = Model(input=inputs, output=conv10)

        model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])
        model = to_multi_gpu(model)
        model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

        return model


    def train(self):

        print("loading data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading data done")
        model = self.get_unet()
        print("got unet")

        model_checkpoint = ModelCheckpoint('sentinesl_unet_roads.hdf5', monitor='loss',verbose=1, save_best_only=True, mode='min')
        print('Fitting model...')
        model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=50, verbose=1, validation_split=0.3, shuffle=True, callbacks=[model_checkpoint])

        
        print('predict test data')
        imgs_mask_test = model.predict(imgs_test, batch_size=8, verbose=1)
        if self.type == 'roads':
            np.save('../results/imgs_mask_test.npy', imgs_mask_test)
        elif self.type == 'cars':
            np.save('../results/imgs_mask_test_cars.npy', imgs_mask_test)

    def save_img(self):

        print("array to image")
        if self.tp == 'roads':
            imgs = np.load('../results/imgs_mask_test.npy')
        elif self.tp == 'cars':
            imgs = np.load('../results/imgs_mask_test_cars.npy')
        for i in range(imgs.shape[0]):
            img = imgs[i]
            img = array_to_img(img)
            img.save("../results/%d.jpg"%(i))

In [13]:
myunet = myUnet('roads')
myunet.train()

loading data
------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------
loading data done


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_normal", padding="same")`
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_normal", padding="same")`
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_initializer="he_normal", padding=

got unet
Fitting model...
Train on 350 samples, validate on 150 samples
Epoch 1/50
350/350 [==============================] - 1213s 3s/step - loss: 7.5754 - binary_crossentropy: 0.0872 - jaccard_coef_int: 0.0999 - val_loss: 7.9113 - val_binary_crossentropy: 0.0125 - val_jaccard_coef_int: 0.1365

Epoch 00001: loss improved from inf to 7.57543, saving model to sentinesl_unet_roads.hdf5
Epoch 2/50
350/350 [==============================] - 1227s 4s/step - loss: 6.8496 - binary_crossentropy: 0.0188 - jaccard_coef_int: 0.0833 - val_loss: 8.0230 - val_binary_crossentropy: 0.0117 - val_jaccard_coef_int: 0.1365

Epoch 00002: loss improved from 7.57543 to 6.84957, saving model to sentinesl_unet_roads.hdf5
Epoch 3/50
280/350 [=======================>......] - ETA: 3:17 - loss: 6.7067 - binary_crossentropy: 0.0202 - jaccard_coef_int: 0.0373

KeyboardInterrupt: 

In [ ]:
# myunet = myUnet('cars')
# myunet.train()
# # myunet.save_img()

**load**

In [ ]:
# # define the checkpoint
# filepath="results/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# # fit the model
# model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

# # load the network weights
# filename = "results/weights-improvement-49-0.4748-bigger.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')